In [ ]:
import os, sys, tarfile
import numpy as np
import csv
import pdb
import data_preprocessing, data_generator, visual, label_parser
from data_preprocessing import RandomShift, RandomHorizontalFlip, Shift
import matplotlib.pyplot as plt
import cv2
import keras
from PIL import Image, ImageDraw
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
TEST_PATH = '../udacity_driving_datasets/'
test_ann = label_parser.parse_annotation_json(TEST_PATH, 'labels_val.json')
print("len(test_ann) = ", len(test_ann))

In [ ]:
size = 300
config_test  = {'grid_w':9, 'grid_h':9, 'img_w' : size, 'img_h' : size, 'is_augment' : True, 'batch_size' : 100}
testGenerator = data_generator.BatchGenerator(test_ann, config_test)

In [ ]:
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['lines.markersize'] = 30
plt.rcParams['lines.linewidth'] = 5

filename = list(test_ann.keys())[2]
print(filename)
rects = test_ann[filename]
image = np.array(Image.open(filename))
plt.imshow(image)
print(rects)
# plt.imshow(image)
plt.imshow(visual.draw_image_rects(image, rects))

In [ ]:
image, rects = testGenerator.get_image_rects(2)
plt.imshow(visual.draw_image_rects(image, rects))

In [ ]:
plt.imshow(visual.draw_image_rects(image, rects))

In [ ]:
print(rects)

In [ ]:
horizontal_flip = RandomHorizontalFlip()
image_aug, rects_aug = horizontal_flip(image, rects)

In [ ]:
plt.imshow(visual.draw_image_rects(image_aug, rects_aug))

In [ ]:
shift = RandomShift(0.2, 0.2)
image_aug, rects_aug = shift(image, rects)
plt.imshow(visual.draw_image_rects(image_aug, rects_aug))

In [ ]:
shift = Shift(0.2, 0.0)
image_aug, rects_aug = shift(image, rects)
plt.imshow(visual.draw_image_rects(image_aug, rects_aug))

In [ ]:
def filter_rects(rects_aug):
    '''
    filter remove rectangles which centers are outside the image
    and cut rectangles which go out the image
    '''
    rects_aug_filtered = []
    for rect in rects_aug:
        x_c = rect[0] + rect[2] / 2.0
        y_c = rect[1] + rect[3] / 2.0
        x_max = rect[0] + rect[2]
        y_max = rect[1] + rect[3]
        w = image.shape[1]
        h = image.shape[0]
        if x_c < w and x_c > 0 and y_c < h  and y_c > 0: # at least half of image must be on image
            x_min = np.maximum(0, rect[0])
            y_min = np.maximum(0, rect[1])
            x_max = np.minimum(w-1, x_max)
            y_max = np.minimum(h-1, y_max)
            rects_aug_filtered += [np.array([x_min, y_min, x_max - x_min, y_max - y_min])]
    return rects_aug_filtered

In [ ]:
shift = Shift(0.0, 0.0)
image_aug, rects_aug = shift(image, rects)
filtered = filter_rects(rects_aug)
plt.imshow(visual.draw_image_rects(image_aug, filtered))

In [ ]:
filtered